In [1]:
from typing import Any, Dict, Optional, Tuple, List, Iterator, Union
import numpy as np

import pandas as pd
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import ConcatDataset, DataLoader, Dataset, Sampler, SequentialSampler #random_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torch import Tensor
import sqlite3
import math
# from torch import default_generator, randperm
# from torch._utils import _accumulate
# from torch.utils.data.dataset import Subset
# from torch_geometric.data import Data
# from torch.utils.data import Sampler

/groups/icecube/moust/miniconda3/envs/icet2/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_hip.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
csv_file = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/upgrade_pure_numu_selection_event_no.csv"
db_path = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/dev_step4_upgrade_028_with_noise_dynedge_pulsemap_v3_merger_aftercrash.db"


In [3]:

def split_csv_based_on_db_rows(db_path, csv_path, int_list, table_name, output_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)

        # Read the CSV file into a pandas DataFrame, considering it doesn't have a header
    df = pd.read_csv(csv_path, ) #header=None

    # The column name of the DataFrame is 0, let's rename it to 'event_number' for clarity
    df.rename(columns={0: 'event_no'}, inplace=True)

    # Count the number of rows for each event_number in the SQLite table
    row_counts = df['event_no'].apply(lambda x: pd.read_sql_query(f"SELECT COUNT(*) FROM {table_name} WHERE event_no = {x}", conn).iloc[0,0])

    # Attach the row counts to the original DataFrame
    df['row_counts'] = row_counts

    # The input list of integers is assumed to be sorted, if not, sort it
    int_list.sort()

    # Create the ranges for splitting the CSV file
    ranges = [(0, int_list[0])] + [(int_list[i-1], int_list[i]) for i in range(1, len(int_list))] + [(int_list[-1], float('inf'))]

    # For each range, select the rows with 'row_counts' within that range and write to a new CSV file
    for i, (lower, upper) in enumerate(ranges):
        df_range = df[(df['row_counts'] >= lower) & (df['row_counts'] < upper)]
        df_range.drop(columns='row_counts').to_csv(f"{output_path}/output_{i+1}.csv", index=False)

    # Close the SQLite connection
    conn.close()

In [4]:
test_csv = "/groups/icecube/moust/storage/cached_event_no/upgrade_pure_numu_selection_event_no_test.csv"
train_csv = "/groups/icecube/moust/storage/cached_event_no/upgrade_pure_numu_selection_event_no_train.csv"
val_csv = "/groups/icecube/moust/storage/cached_event_no/upgrade_pure_numu_selection_event_no_val.csv"

In [8]:
df = pd.read_csv(val_csv)

In [9]:
df

,Unnamed: 0,CascadeFilter_13,DeepCoreFilter_13,EventID,L3_oscNext_bool,L4_oscNext_bool,L5_oscNext_bool,L6_oscNext_bool,L7_oscNext_bool,MuonFilter_13,...,inelasticity,interaction_type,pid,position_x,position_y,position_z,sim_type,stopped_muon,track_length,zenith
0,10,-1.0,1.0,36.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.290624,1.0,14.0,-6.710774,-62.118681,-189.266099,genie,-1.0,-1.0,1.840785
1,22,-1.0,0.0,30.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.319193,1.0,-14.0,64.901104,-139.468445,-519.591668,genie,-1.0,-1.0,0.916502
2,27,-1.0,1.0,30.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.167333,1.0,-14.0,93.307056,-127.478072,-418.871037,genie,-1.0,-1.0,2.416366
3,35,-1.0,1.0,115.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.062827,1.0,14.0,53.785919,-59.037806,-323.476400,genie,-1.0,-1.0,1.501030
4,45,-1.0,1.0,98.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.441630,1.0,-14.0,96.607002,-179.181420,-486.508775,genie,-1.0,-1.0,1.104446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102022,716640,-1.0,0.0,3050.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.032871,1.0,-14.0,200.529404,-2.128855,-343.039218,genie,-1.0,-1.0,1.433850
102023,716644,-1.0,1.0,3076.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.100296,1.0,-14.0,139.997070,-138.179009,-333.642152,genie,-1.0,-1.0,0.846469
102024,716647,-1.0,1.0,3084.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.426457,1.0,-14.0,-66.024897,-61.042635,-242.314790,genie,-1.0,-1.0,1.772950
102025,716650,-1.0,1.0,3099.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.074860,1.0,-14.0,222.274117,-125.957383,-334.514415,genie,-1.0,-1.0,1.969339


In [11]:
# split_csv_based_on_db_rows(
#     db_path,
#     train_csv, 
#     [5,10,20,40,100,200], 
#     "SplitInIcePulses_dynedge_v2_Pulses", 
#     "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train")
# split_csv_based_on_db_rows(
#     db_path,
#     test_csv, 
#     [5,10,20,40,100,200], 
#     "SplitInIcePulses_dynedge_v2_Pulses", 
#     "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test")
split_csv_based_on_db_rows(
    db_path,
    val_csv, 
    [5,10,20,40,100,200], 
    "SplitInIcePulses_dynedge_v2_Pulses", 
    "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val")

In [9]:
df = pd.read_csv(csv_file,)
df.head()

,6
0,20
1,28
2,32
3,46
4,54


In [14]:
split_csv_based_on_db_rows(db_path, csv_file, [20,100], "SplitInIcePulses_dynedge_v2_Pulses", "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/upgrade_pure_numu_selection_event_no_split")

In [5]:
import os
import sqlite3
import numpy as np
import pandas as pd
from scipy.stats import expon

# Function to generate exponential distribution
def generate_exponential_distribution(n, scale=50, size=1):
    return expon.rvs(scale=scale, size=size)

# Create a SQLite database in a file
db_filename = "events.db"
conn = sqlite3.connect(db_filename)
c = conn.cursor()

# # Create tables
# c.execute("CREATE TABLE truth (event_no INTEGER PRIMARY KEY, energy REAL, inelasticity REAL)")
# c.execute("CREATE TABLE pulsemap (event_no INTEGER, dom_x REAL, dom_y REAL, dom_z REAL, time REAL, charge REAL)")

# # Populate tables
# np.random.seed(0)
# num_events = 10000

# # Generate event numbers
# event_nos = np.arange(num_events)

# # Generate energies and inelasticities
# energies = np.random.uniform(size=num_events)
# inelasticities = np.random.uniform(size=num_events)

# # Populate truth table
# for event_no, energy, inelasticity in zip(event_nos, energies, inelasticities):
#     # Convert numpy int64 to native int
#     event_no = int(event_no)
#     c.execute("INSERT INTO truth VALUES (?, ?, ?)", (event_no, energy, inelasticity))

# # Generate pulsemap data
# for event_no in event_nos:
#     num_rows = int(generate_exponential_distribution(n=1, scale=30, size=1))
#     for _ in range(num_rows):
#         dom_x, dom_y, dom_z, time, charge = np.random.uniform(size=5)
#         # Convert numpy int64 to native int
#         event_no = int(event_no)
#         c.execute("INSERT INTO pulsemap VALUES (?, ?, ?, ?, ?, ?)", (event_no, dom_x, dom_y, dom_z, time, charge))

# # Commit changes
# conn.commit()

# Create histogram of the number of hits in the events
c.execute("SELECT event_no, COUNT(*) FROM pulsemap GROUP BY event_no")
data = c.fetchall()
event_nos, counts = zip(*data)

# Divide event_no's into four groups based on the number of rows with the same event_no
counts = np.array(counts)
event_nos = np.array(event_nos)
indices = counts.argsort()
sorted_event_nos = event_nos[indices]
group1, group2, group3, group4 = np.array_split(sorted_event_nos, 4)

# Write event_no's for each group to a separate CSV file
# os.makedirs("/mnt/data/event_groups", exist_ok=True)
np.savetxt("/mnt/data/event_groups/group1.csv", group1, delimiter=",", fmt="%d")
np.savetxt("/mnt/data/event_groups/group2.csv", group2, delimiter=",", fmt="%d")
np.savetxt("/mnt/data/event_groups/group3.csv", group3, delimiter=",", fmt="%d")
np.savetxt("/mnt/data/event_groups/group4.csv", group4, delimiter=",", fmt="%d")

db_filename, "/mnt/data/event_groups/group1.csv", "/mnt/data/event_groups/group2.csv", "/mnt/data/event_groups/group3.csv", "/mnt/data/event_groups/group4.csv"


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/event_groups/group1.csv'

In [21]:
csv_file = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/upgrade_pure_numu_selection_event_no.csv"
db_path = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/dev_step4_upgrade_028_with_noise_dynedge_pulsemap_v3_merger_aftercrash.db"
df = pd.read_csv(csv_file)
print(len(df))
csv_filenames = [
    "/groups/icecube/moust/storage/cached_event_no/output_1.csv",
    "/groups/icecube/moust/storage/cached_event_no/output_2.csv", 
    "/groups/icecube/moust/storage/cached_event_no/output_3.csv"
    ]
event_numbers = []
for csv_filename in csv_filenames:
    df = pd.read_csv(csv_filename)
    event_numbers.extend(df['event_no'].tolist())
len(event_numbers)

1098831


1098832

In [4]:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT event_no, COUNT(*) FROM SplitInIcePulses_dynedge_v2_Pulses GROUP BY event_no")
data = c.fetchall()

In [7]:
data

AttributeError: 'list' object has no attribute 'shape'

In [11]:
query_all ="SELECT * FROM truth"
with sqlite3.connect(db_path) as conn:
    db_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type = 'table'", conn)
    print('Tables within the SQL database is:')
    print(db_tables)
    print()
    mini_db = {name:  pd.read_sql_query(f"SELECT * FROM {name} LIMIT 3000", conn) for name in db_tables.name}

Tables within the SQL database is:
                                       name
0                                     truth
1                         pisa_dependencies
2                          SplitInIcePulses
3               SplitInIcePulses_TruthFlags
4                   SplitIceCubePulsesTWSRT
5        SplitIceCubePulsesTWSRT_TruthFlags
6         SplitInIcePulses_GraphSage_Pulses
7    SplitInIcePulses_GraphSage_Predictions
8   SplitInIcePulses_dynedge_v2_Predictions
9        SplitInIcePulses_dynedge_v2_Pulses
10    no_upgrade_strings_dynedge_v2_cleaned



In [3]:
class ChunkDataset(Dataset):
    def __init__(self, db_filename, csv_filenames):
        # self.conn = sqlite3.connect(db_filename)
        # self.c = self.conn.cursor()
        # self.c.execute("SELECT event_no FROM truth")
        # self.event_nos = [row[0] for row in self.c.fetchall()]
        self.conn = sqlite3.connect(db_filename)
        self.c = self.conn.cursor()
        self.event_nos = []
        for csv_filename in csv_filenames:
            df = pd.read_csv(csv_filename)
            self.event_nos.extend(df['event_no'].tolist())

    def __len__(self):
        return len(self.event_nos)

    def __getitem__(self, idx):
        event_no = self.event_nos[idx]
        self.c.execute("SELECT energy FROM truth WHERE event_no = ?", (event_no,))
        energy = self.c.fetchone()[0]
        self.c.execute("SELECT dom_x, dom_y, dom_z, dom_time, charge FROM SplitInIcePulses_dynedge_v2_Pulses WHERE event_no = ?", (event_no,))
        pulsemap_data = self.c.fetchall()
        return torch.tensor(energy, dtype=torch.float32), torch.tensor(pulsemap_data, dtype=torch.float32)
    
class ChunkSampler(Sampler):
    def __init__(self, csv_filenames, batch_sizes):
        self.event_nos = []
        for csv_filename, batch_size in zip(csv_filenames, batch_sizes):
            # event_nos = np.loadtxt(csv_filename, delimiter=",", dtype=int).tolist()
            event_nos = pd.read_csv(csv_filename)['event_no'].tolist()
            self.event_nos.extend([event_nos[i:i + batch_size] for i in range(0, len(event_nos), batch_size)])

    def __iter__(self):
        return iter(self.event_nos)

    def __len__(self):
        return len(self.event_nos)
    
def collate_fn(data):
    # Split the data into energies and pulsemap data
    energies, pulsemap_data = zip(*data)
    
    # Pad the pulsemap data
    pulsemap_data = pad_sequence(pulsemap_data, batch_first=True)
    
    return torch.stack(energies), pulsemap_data

csv_filenames = [
    "/groups/icecube/moust/storage/cached_event_no/output_1.csv",
    "/groups/icecube/moust/storage/cached_event_no/output_2.csv", 
    "/groups/icecube/moust/storage/cached_event_no/output_3.csv"
    ]

# csv_file = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/upgrade_pure_numu_selection_event_no.csv"
db_path = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/dev_step4_upgrade_028_with_noise_dynedge_pulsemap_v3_merger_aftercrash.db"

# Test the EventDataset
dataset = ChunkDataset(db_path, csv_filenames ) #db_filename
batch_sizes = [100, 50, 10]  # Adjust these batch sizes according to your machine's memory limits
sampler = ChunkSampler(csv_filenames, batch_sizes)
dataloader = DataLoader(dataset, batch_sampler=sampler, collate_fn=collate_fn, num_workers=4)

print(dataset[0])  # Print the first sample from the dataset
print(len(dataset))  # Print the total number of samples in the dataset

(tensor(24.1367), tensor([[ 1.9434e+02, -3.0920e+01, -2.1078e+02,  1.1114e+04,  1.1386e+00],
        [ 9.0490e+01,  8.2350e+01, -1.6214e+02,  1.0005e+04,  4.8686e-01],
        [ 9.0490e+01,  8.2350e+01, -1.9619e+02,  9.8812e+03,  3.0716e-01],
        [ 9.0490e+01,  8.2350e+01, -2.1321e+02,  1.0058e+04,  8.5537e-01],
        [ 9.0490e+01,  8.2350e+01, -2.3023e+02,  9.9392e+03,  1.3410e+00],
        [ 9.0490e+01,  8.2350e+01, -2.3023e+02,  9.9442e+03,  1.1514e+00],
        [ 9.0490e+01,  8.2350e+01, -2.3023e+02,  9.9967e+03,  2.4345e-01],
        [ 9.0490e+01,  8.2350e+01, -2.4725e+02,  9.9892e+03,  8.4372e-01],
        [ 1.3203e+02,  2.0298e+02, -1.7809e+02,  1.0410e+04,  5.9601e-01],
        [ 4.1600e+01,  3.5490e+01, -2.4901e+02,  1.0430e+04,  4.9121e-01],
        [ 1.0694e+02,  2.7090e+01, -1.8880e+02,  1.0353e+04,  7.8922e-01],
        [ 1.1319e+02, -6.0470e+01, -3.7837e+02,  1.1606e+04,  6.4532e-01],
        [ 2.6957e+01, -3.1191e+01, -3.1668e+02,  1.0851e+04,  1.0357e+00]]))
10988

In [17]:
event_numbers = []
for csv_filename in csv_filenames:
    df = pd.read_csv(csv_filename)
    event_numbers.extend(df['event_no'].tolist())

In [24]:


class EventSampler(Sampler):
    def __init__(self, csv_filenames, batch_sizes):
        self.event_nos = []
        for csv_filename, batch_size in zip(csv_filenames, batch_sizes):
            # event_nos = np.loadtxt(csv_filename, delimiter=",", dtype=int).tolist()
            event_nos = pd.read_csv(csv_filename)['event_no'].tolist()
            self.event_nos.extend([event_nos[i:i + batch_size] for i in range(0, len(event_nos), batch_size)])

    def __iter__(self):
        return iter(self.event_nos)

    def __len__(self):
        return len(self.event_nos)
    
def collate_fn(data):
    # Split the data into energies and pulsemap data
    energies, pulsemap_data = zip(*data)
    
    # Pad the pulsemap data
    pulsemap_data = pad_sequence(pulsemap_data, batch_first=True)
    
    return torch.stack(energies), pulsemap_data

csv_filenames = [
    "/groups/icecube/moust/storage/cached_event_no/output_1.csv",
    "/groups/icecube/moust/storage/cached_event_no/output_2.csv", 
    "/groups/icecube/moust/storage/cached_event_no/output_3.csv"
    ]

batch_sizes = [100, 50, 10]  # Adjust these batch sizes according to your machine's memory limits
sampler = EventSampler(csv_filenames, batch_sizes)
print(len(sampler))  # Print the total number of batches in the sampler
# print(sampler[0])  # Print the first batch from the sampler


37930


TypeError: 'EventSampler' object is not subscriptable

In [25]:
def collate_fn(data):
    # Split the data into energies and pulsemap data
    energies, pulsemap_data = zip(*data)
    
    # Pad the pulsemap data
    pulsemap_data = pad_sequence(pulsemap_data, batch_first=True)
    
    return torch.stack(energies), pulsemap_data

# Test the collate function with the first two samples from the dataset
collate_fn([dataset[0], dataset[1]])

(tensor([24.1367, 18.3989]),
 tensor([[[ 1.9434e+02, -3.0920e+01, -2.1078e+02,  1.1114e+04,  1.1386e+00],
          [ 9.0490e+01,  8.2350e+01, -1.6214e+02,  1.0005e+04,  4.8686e-01],
          [ 9.0490e+01,  8.2350e+01, -1.9619e+02,  9.8812e+03,  3.0716e-01],
          [ 9.0490e+01,  8.2350e+01, -2.1321e+02,  1.0058e+04,  8.5537e-01],
          [ 9.0490e+01,  8.2350e+01, -2.3023e+02,  9.9392e+03,  1.3410e+00],
          [ 9.0490e+01,  8.2350e+01, -2.3023e+02,  9.9442e+03,  1.1514e+00],
          [ 9.0490e+01,  8.2350e+01, -2.3023e+02,  9.9967e+03,  2.4345e-01],
          [ 9.0490e+01,  8.2350e+01, -2.4725e+02,  9.9892e+03,  8.4372e-01],
          [ 1.3203e+02,  2.0298e+02, -1.7809e+02,  1.0410e+04,  5.9601e-01],
          [ 4.1600e+01,  3.5490e+01, -2.4901e+02,  1.0430e+04,  4.9121e-01],
          [ 1.0694e+02,  2.7090e+01, -1.8880e+02,  1.0353e+04,  7.8922e-01],
          [ 1.1319e+02, -6.0470e+01, -3.7837e+02,  1.1606e+04,  6.4532e-01],
          [ 2.6957e+01, -3.1191e+01, -3.1668e+0

In [26]:
batch_sizes = [100, 50, 10]  # Adjust these batch sizes according to your machine's memory limits
sampler = EventSampler(csv_filenames, batch_sizes)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_sampler=sampler, collate_fn=collate_fn, num_workers=4)


In [32]:
for energies, pulsemap_data in dataloader:
    print(energies.shape)
    print(pulsemap_data.shape)
    break


torch.Size([100])
torch.Size([100, 19, 5])


In [21]:
class ChunkDataset(Dataset):
    """
    PyTorch dataset for loading chunked data from an SQLite database.
    This dataset retrieves pulsemap and truth data for each event from the database.

    Args:
        db_filename (str): Filename of the SQLite database.
        csv_filenames (list of str): List of CSV filenames containing event numbers.
        pulsemap_table (str): Name of the table containing pulsemap data.
        truth_table (str): Name of the table containing truth data.
        truth_variable (str): Name of the variable to query from the truth table.
        feature_variables (list of str): List of variable names to query from the pulsemap table.
    """

    def __init__(
        self,
        db_path: str,
        csv_filenames: List[str],
        pulsemap_table: str,
        truth_table: str,
        truth_variable: str,
        feature_variables: List[str]
    ) -> None:
        self.conn = sqlite3.connect(db_path)  # Connect to the SQLite database
        self.c = self.conn.cursor()
        self.event_nos = []
        for csv_filename in csv_filenames:
            df = pd.read_csv(csv_filename)
            self.event_nos.extend(df['event_no'].tolist())  # Collect event numbers from CSV files
        self.pulsemap_table = pulsemap_table  # Name of the table containing pulsemap data
        self.truth_table = truth_table  # Name of the table containing truth data
        self.truth_variable = truth_variable  # Name of the variable to query from the truth table
        self.feature_variables = feature_variables  # List of variable names to query from the pulsemap table

    def __len__(self) -> int:
        return len(self.event_nos)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        event_no = self.event_nos[idx]
        # Query the truth variable for the given event number
        self.c.execute(f"SELECT {self.truth_variable} FROM {self.truth_table} WHERE event_no = ?", (event_no,))
        truth_value = self.c.fetchone()[0]
        feature_query = ', '.join(self.feature_variables)
        # Query the feature variables from the pulsemap table for the given event number
        self.c.execute(f"SELECT {feature_query} FROM {self.pulsemap_table} WHERE event_no = ?", (event_no,))
        pulsemap_data = self.c.fetchall()
        return torch.tensor(truth_value, dtype=torch.float32), torch.tensor(pulsemap_data, dtype=torch.float32)
    
    def close_connection(self) -> None:
        self.conn.close()


class ChunkSampler(Sampler):
    """
    PyTorch sampler for creating chunks from event numbers.

    Args:
        csv_filenames (List[str]): List of CSV filenames containing event numbers.
        batch_sizes (List[int]): List of batch sizes for each CSV file.
    """

    def __init__(self, csv_filenames: List[str], batch_sizes: List[int]) -> None:
        self.event_nos = []
        for csv_filename, batch_size in zip(csv_filenames, batch_sizes):
            event_nos = pd.read_csv(csv_filename)['event_no'].tolist()
            self.event_nos.extend([event_nos[i:i + batch_size] for i in range(0, len(event_nos), batch_size)])

    def __iter__(self) -> Iterator:
        return iter(self.event_nos)

    def __len__(self) -> int:
        return len(self.event_nos)


def collate_fn(data: List[Tuple[torch.Tensor, torch.Tensor]]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    # Split the data into truths and pulsemap data
    truths, pulsemap_data = zip(*data)

    # Pad the pulsemap data
    pulsemap_lengths = [len(x) for x in pulsemap_data]
    pulsemap_data = pad_sequence(pulsemap_data, batch_first=True, padding_value=0)

    pad_mask = torch.zeros_like(pulsemap_data[:, :, 0]).type(torch.bool)

    for i, length in enumerate(pulsemap_lengths):
        pad_mask[i, length:] = True

    return (pulsemap_data, torch.stack(truths), pad_mask)

In [11]:
chunk_csv_train = [
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_1.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_2.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_3.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_4.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_5.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_6.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/train/output_7.csv",
]
chunk_csv_test = [
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_1.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_2.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_3.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_4.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_5.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_6.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/test/output_7.csv",
]
chunk_csv_val = [
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_1.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_2.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_3.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_4.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_5.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_6.csv",
  "/groups/icecube/moust/storage/cached_event_no/upgrade_numu/val/output_7.csv",
]

batch_sizes = [512, 256, 128, 64, 32, 16, 8]
truth_table = "truth"
db_path = "/groups/icecube/petersen/GraphNetDatabaseRepository/Upgrade_Data/sqlite3/dev_step4_upgrade_028_with_noise_dynedge_pulsemap_v3_merger_aftercrash.db"
pulsemap = "SplitInIcePulses_dynedge_v2_Pulses"
input_cols =  ["dom_x", "dom_y", "dom_z", "dom_time", "charge"]
target_cols = "inelasticity"

In [22]:
data_train = ChunkDataset(
    db_path = db_path,
    csv_filenames = chunk_csv_train,
    pulsemap_table = pulsemap, 
    truth_table = truth_table, 
    feature_variables = input_cols, 
    truth_variable = target_cols, 
    )
# data_test = ChunkDataset(db_path, pulsemap, truth_table, input_cols, target_cols, chunk_csv_test)
# data_val = ChunkDataset(db_path, pulsemap, truth_table, input_cols, target_cols, chunk_csv_val)


In [25]:
dataloader_train = DataLoader(
    dataset = data_train,
    collate_fn=collate_fn,
    batch_sampler=ChunkSampler(chunk_csv_train, batch_sizes),
    )

    

In [26]:
for i, (features, _, _) in enumerate(dl):
    print(features.shape)
    break

torch.Size([256, 39, 5])
torch.Size([256])


In [29]:
iter(dataloader_train)